In [ ]:
!git clone https://github.com/threestudio-project/threestudio.git

In [ ]:
!sudo apt-get update
!sudo apt-get install mesa-common-dev libegl1-mesa-dev -y
!sudo apt-get install ninja-build  -y

In [ ]:
%pip install --upgrade pip

In [ ]:
%cd /notebooks/threestudio

In [ ]:
%pip install ninja

In [ ]:
""" %pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 """
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 

In [ ]:
%pip install  -r requirements.txt  --upgrade --force-reinstall

In [ ]:
%pip install lightning==2.0.0 omegaconf==2.3.0 jaxtyping typeguard diffusers transformers accelerate opencv-python tensorboard matplotlib imageio imageio[ffmpeg] trimesh bitsandbytes sentencepiece safetensors huggingface_hub libigl xatlas 
%pip install open3d plotly # mesh visualization

In [ ]:
%pip install git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2
%pip install git+https://github.com/NVlabs/nvdiffrast.git
%pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

In [ ]:
!mkdir load/zero123
%cd load/zero123
!wget https://zero123.cs.columbia.edu/assets/zero123-xl.ckpt
%cd ../../

In [ ]:
# PHASE 1
%run launch.py  --config configs/zero123.yaml --train --gpu 0 data.image_path="/notebooks/stable-dreamfusion/side_rgba.png" name=dragon_s tag=Phase1_34 data.batch_size=4 data.default_azimuth_deg=90 data.default_elevation_deg=0  system.freq.guidance_eval=37


In [ ]:
# SKIP
%run launch.py  --config configs/zero123.yaml --train --gpu 0 data.image_path=/notebooks/stable-dreamfusion/front_rgba.png name=dragon_side tag=Phase1_Front data.batch_size=4  resume="/notebooks/threestudio/outputs/dragon_side/Phase1_Side@20230716-044416/ckpts/last.ckpt" trainer.max_steps=600


In [ ]:
# Phase 1.5 - 512 refine
%run launch.py --config configs/zero123-geometry.yaml --train --gpu 0 system.geometry_convert_from="/notebooks/threestudio/outputs/dragon_s/Phase1_34@20230716-083702/ckpts/last.ckpt" data.image_path="/notebooks/stable-dreamfusion/34_rgba.png"  data.batch_size=4 data.default_azimuth_deg=45 data.default_elevation_deg=45  system.exporter.context_type=cuda  use_timestamp=False name=dragon_34 tag=Phase1p5 


In [ ]:
# Phase 2 - dreamfusion
%run launch.py --config configs/experimental/imagecondition_zero123nerf.yaml --train --gpu 0 data.image_path="/notebooks/stable-dreamfusion/front_rgba.png" system.prompt_processor.prompt="A 3D model of a chibi dragon shaped figure" system.weights="/notebooks/threestudio/outputs/dragon_s/Phase1_34@20230716-083702/ckpts/last.ckpt" name=dragon_s tag=Phase2t

In [ ]:
# Phase 2 - SDF + dreamfusion
%run launch.py --config configs/experimental/imagecondition_zero123nerf_refine.yaml --train --gpu 0  data.image_path="/notebooks/stable-dreamfusion/side_rgba.png" system.prompt_processor.prompt="A 3D model of a chibi dragon shaped figure" system.geometry_convert_from="/notebooks/threestudio/outputs/dragon_s/Phase1_34@20230716-083702/ckpts/last.ckpt" name=dragon_s tag=Phase2_refine  data.default_azimuth_deg=0 data.default_elevation_deg=0  data.default_camera_distance=3.8 data.default_fovy_deg=20.0

OTHER

In [ ]:
#EXPORT
%run launch.py --config "outputs/dragon_34/Phase2_refine@20230716-081539/configs/parsed.yaml" --export --gpu 0 resume="outputs/dragon_34/Phase2_refine@20230716-081539/ckpts/last.ckpt" system.exporter.context_type=cuda  system.exporter_type=mesh-exporter

pip install 'mediapipe'
sudo apt-get update && apt-get install -y --no-install-recommends \
    pkg-config \
    libglvnd0 \
    libgl1 \
    libglx0 \
    libegl1 \
    libgles2 \
    libglvnd-dev \
    libgl1-mesa-dev \
    libegl1-mesa-dev \
    libgles2-mesa-dev \
    cmake \
    curl \
    libsm6 \
    libxext6 \
    libxrender-dev
sudo apt-get install libnvidia-gl-418-server
sudo apt-get install  libnvidia-gl 
sudo apt-get update
sudo apt-get install mesa-common-dev libegl1-mesa-dev
sudo apt-get install ninja-build
source myenv/bin/activate
python3.10 -m ipykernel install  --name=myenv
python3.10 launch.py --config "outputs/zero123/[64, 128, 256]_front_rgba.png_prog0@20230716-010647/configs/parsed.yaml" --export --gpu 0 resume="outputs/zero123/[64, 128, 256]_front_rgba.png_prog0@20230716-010647/ckpts/last.ckpt" system.exporter_type=mesh-exporter

